In [23]:
import pandas as pd
from commcare_export.checkpoint import CheckpointManagerWithDetails
from commcare_export.commcare_hq_client import CommCareHqClient, AUTH_MODE_APIKEY
from commcare_export.commcare_minilinq import get_paginator, PaginationMode
from datetime import datetime
import re
import  numpy as np
# CommCare credentials
username = 'amutenha@zimttech.org'  # Replace with your actual CommCare username
domain = 'zdip-qa'  # Your CommCare project domain
hq_host = 'https://zdip.zimttech.org'  # Your CommCare HQ custom host
API_KEY = '92fbb3f98433a6929a33015273ebe7b38b6dac39'  # Replace with the API key generated in CommCare


zazicSites= pd.read_excel('zazicFacilities.xlsx')



data = pd.ExcelFile('ZDIPlookup.xlsx')

iqaLookup = pd.read_excel('iqaLookup.xlsx')

# lookupTable = pd.read_excel(data,'previousVMMCSubmissions')

# lookupTable.columns = [col.replace('field: ', '') if col.startswith('field: ') else col for col in lookupTable.columns]

# Initialize API client
api_client = CommCareHqClient(hq_host, domain, username, API_KEY, AUTH_MODE_APIKEY)

# Create a paginator for the 'form' resource (fetching data from a form)
form_paginator = get_paginator(resource='form', pagination_mode=PaginationMode.date_modified, page_size=100)
form_paginator.init()

# Initialize checkpoint manager (None used here for simplicity)
checkpoint_manager = CheckpointManagerWithDetails(None, None, PaginationMode.date_modified)

# Date range for filtering
start_date = datetime(2025,6,30)  # 2 December 2024
end_date = datetime(2025, 12,30)  # 6 December 2024

# Specify the app_id to filter forms
desired_app_id ='0778c330a59625e988e36d5a7d0386b6'  


referrals_data = []
AEs_data = []
general_data = []

# Fetch forms iteratively
# Fetch forms using manual pagination
all_forms = []
offset = 0

params = {
    'limit': 100000,
    'app_id': desired_app_id,
    'received_on_start': start_date.strftime('%Y-%m-%dT%H:%M:%S'),
    'received_on_end': end_date.strftime('%Y-%m-%dT%H:%M:%S')
}

forms = api_client.iterate(
    'form',
    form_paginator,
    checkpoint_manager=checkpoint_manager,
    params=params
)

for form in forms:
    all_forms.append(form)
    if len(all_forms) >= 100000:  # Early stopping if desired
        break

In [24]:
# Print a few fetched forms to verify filtering
for form in all_forms[:5]:  
    pass
    # print(f"Received On: {form.get('received_on')}, App ID: {form.get('app_id')}")

filteredResults =[]
for i in all_forms:
    if i["app_id"] ==desired_app_id:
        filteredResults.append(i)

print(filteredResults)
# Filter and categorize forms by app_id and date
for form in forms:
    if form.get('app_id') == desired_app_id:
        # print(form)
        form_received_date = datetime.strptime(form['received_on'], "%Y-%m-%dT%H:%M:%S.%fZ")
        if start_date <= form_received_date <= end_date:
            general_data.append(form)
# Convert each list to a pandas DataFrame
general_df = pd.json_normalize(general_data)
general_df.to_excel('test.xlsx')

general_df.columns = [re.split(r'[./]', col)[-1] for col in general_df.columns]
desiredColumns = [re.split(r'[./]', col)[-1] for col in iqaLookup.columns]
print("desiredcolumns")
print(desiredColumns)

[{'app_id': '0778c330a59625e988e36d5a7d0386b6', 'archived': False, 'attachments': {'form.xml': {'content_type': 'text/xml', 'length': 3505, 'url': 'https://zdip.zimttech.org/a/zdip-qa/api/form_attachment/v1/c60eec85-9970-4d06-948f-cf2994caf8c0/form.xml'}}, 'build_id': '67775be853d348e79ec9f44c924e7a1d', 'domain': 'zdip-qa', 'edited_by_user_id': None, 'edited_on': None, 'form': {'#type': 'data', '@name': 'ZAZIC OSDV', '@uiVersion': '1', '@version': '669', '@xmlns': 'http://openrosa.org/formdesigner/72526D77-6087-4854-811D-97DBBA955075', 'activity_and_time_period_under_review': {'section': 'd'}, 'ending_month_to_numeric': '0', 'meta': {'@xmlns': 'http://openrosa.org/jr/xforms', 'appVersion': 'CommCare Android, version "2.56.1"(471796). App v669. CommCare Version 2.56.1. Build 471796, built on: 2025-05-12', 'app_build_version': 669, 'commcare_version': '2.56.1', 'deviceID': 'commcare_51deb0f1-b0d0-487f-bc8a-4479d5af3f73', 'drift': '0', 'geo_point': None, 'instanceID': 'c60eec85-9970-4d06-

In [25]:
# New columns that should NOT be deleted
columns_to_retain = desiredColumns
# Columns that must be numeric (including the new ones)
numeric_columns = [
    
]

# general_df['AERecordingYear'] = pd.to_numeric(general_df['AERecordingYear'])
# Combine the two lists (retain columns that are in either columns_to_retain or numeric_columns)
# columns_to_keep = list(set(columns_to_retain + numeric_columns))
columns_to_keep = columns_to_retain
# Identify missing columns
missing_columns = [col for col in columns_to_keep if col not in general_df.columns]

In [26]:
# Add missing columns to the DataFrame and assign zero
for col in missing_columns:
    general_df[col] = np.nan
# Drop the columns that are not in the columns_to_keep list
# Sort the columns alphabetically in both DataFrames

# For the 'general_df', sort the columns alphabetically
general_df_sorted = general_df[columns_to_keep]

# For the AE-specific columns, we can do the same
# ae_columns = [
#     'aeComments', 'ae_type_code', 'date_ae_identified', 'AERecordingYear', 'AERecordingSite', 
#     'client_age', 'circumcising_cadre', 'ae_classification', 'AERecordingMonth', 'mcMethod','vmmc_number'
# ]


# referrals_columns =['ReferrelRecordingMonth', 
#     'ReferrelRecordingSite', 'ReferrelRecordingYear', 'services_referred','reason_for_referral']


# other_referrals = []
# ae_data_df_sorted = general_df[ae_columns][sorted(ae_columns)]
# ae_data_df_sorted = ae_data_df_sorted[ae_data_df_sorted['AERecordingMonth'].notna()]

# referralsDF_sorted = general_df[referrals_columns][sorted(referrals_columns)]
# referralsDF_sorted = referralsDF_sorted[referralsDF_sorted['ReferrelRecordingMonth'].notna()]
general_df_sorted.to_excel("data.xlsx")

C:\Users\AshleyMutenha\AppData\Local\Temp\ipykernel_23376\1743773583.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  general_df[col] = np.nan
C:\Users\AshleyMutenha\AppData\Local\Temp\ipykernel_23376\1743773583.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  general_df[col] = np.nan
C:\Users\AshleyMutenha\AppData\Local\Temp\ipykernel_23376\1743773583.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

In [27]:
general_df_sorted

,section,District,facility,startingMonth,startingYear,endingMonth,endingYear,clientSpaceSize,spaceCleaniness,waitingSpace,...,starting_month_to_numeric,ending_month_to_numeric,completed_time,started_time,username,username,received_on,form_link,hq_user,month_year_repeat_group
0,d,MARONDERA,Dombotombo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,jgavhera,jgavhera,2025-06-30T12:32:34.958766Z,NaN,NaN,NaN
1,e,MARONDERA,Dombotombo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,jgavhera,jgavhera,2025-06-30T11:59:36.289790Z,NaN,NaN,NaN
2,b,MARONDERA,Dombotombo,October,2024,May,2025,NaN,NaN,NaN,...,10,5,NaN,NaN,amutenha,amutenha,2025-06-30T11:51:05.207097Z,NaN,NaN,"[{'calculatedMonth': 'October', 'calculatedYea..."
3,c,SANYATI,Jompani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,fgwenzi,fgwenzi,2025-06-30T06:18:41.783379Z,NaN,NaN,NaN
4,c,MBIRE,Angwa Clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,amutenha,amutenha,2025-06-30T05:42:07.558273Z,NaN,NaN,NaN
